In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

In [46]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    try:
        title = json_data['headline']
        date = json_data['datePublished']
        date_m = json_data['dateModified']
        genre = json_data.get('genre', [])
        keywords = json_data.get('keywords', [])
    except:
        title = soup.find('h1', class_='content--title').text
        date = soup.find_all('time')[-1].get('datetime')
        date_m = ''
        keywords = []
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic not in data:
            data.append(dic)
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [53]:
htmls = get_link(0); htmls

['https://www3.nhk.or.jp/news/html/20181212/k10011744981000.html',
 'https://www3.nhk.or.jp/news/html/20181213/k10011746391000.html',
 'https://www3.nhk.or.jp/news/html/20181214/k10011746561000.html',
 'https://www3.nhk.or.jp/news/html/20181214/k10011747261000.html',
 'https://www3.nhk.or.jp/news/html/20181215/k10011748011000.html',
 'https://www3.nhk.or.jp/news/html/20181215/k10011748121000.html',
 'https://www3.nhk.or.jp/news/html/20181216/k10011749001000.html',
 'https://www3.nhk.or.jp/news/html/20181217/k10011749591000.html',
 'https://www3.nhk.or.jp/news/html/20181217/k10011749751000.html',
 'https://www3.nhk.or.jp/news/html/20181217/k10011749811000.html',
 'https://www3.nhk.or.jp/news/html/20181218/k10011750661000.html',
 'https://www3.nhk.or.jp/news/html/20181218/k10011750781000.html',
 'https://www3.nhk.or.jp/news/html/20181218/k10011750821000.html',
 'https://www3.nhk.or.jp/news/html/20181218/k10011751011000.html',
 'https://www3.nhk.or.jp/news/html/20181218/k10011751481000.ht

# scrape

In [50]:
driver = webdriver.Chrome()

In [48]:
for nhkurl in tqdm.tqdm(htmls[:-2]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')

    dic = scrape(html)

    js(dic)





  0%|          | 0/265 [00:00<?, ?it/s]



  0%|          | 1/265 [00:52<3:50:01, 52.28s/it]



  1%|          | 2/265 [02:10<4:22:45, 59.95s/it]



  1%|          | 3/265 [03:25<4:42:29, 64.69s/it]



  2%|▏         | 4/265 [04:41<4:55:32, 67.94s/it]



  2%|▏         | 5/265 [05:59<5:07:31, 70.97s/it]

KeyboardInterrupt: 

In [47]:
html = driver.page_source.encode('utf-8')
dic = scrape(html)
dic

{'id': 'k10011742251000',
 'title': '外国人の銀行口座を数日で開設へ セブン銀行が新サービス',
 'article': '日本で暮らす外国人が増えることが見込まれるとして、コンビニＡＴＭを展開するセブン銀行は、ベンチャー企業との連携でこれまでより速く銀行口座を作れるサービスを始めることになりました。\nこのサービスは、セブン銀行が、ネット上でアジアなどの外国人向けにビザの申請サービスを手がけるベンチャー企業「ｏｎｅｖｉｓａ」と連携して来年春にも始めます。外国人が日本で銀行口座を作る場合、在留資格や収入などを確認するため、来日してから半年ほどかかることもあるということです。このため両社は、来日前、ビザを申請する際の情報を共有してあらかじめ信用性などを判断し、来日後、数日程度で口座を持てるようにすることを目指すとしています。外国人材の受け入れを拡大する改正出入国管理法の成立で、今後、国内で暮らす外国人が増えることが見込まれるとして、銀行は、母国への送金などで手数料収入の増加につなげたいねらいです。10日記者会見したセブン銀行の松橋正明専務執行役員は「来日してすぐに最低限の金融サービスを受けられる環境を整えたい」と話していました。',
 'genre': ['ビジネス'],
 'keywords': [],
 'url': 'https://www3.nhk.or.jp/news/html/20181210/k10011742251000.html',
 'datePublished': '2018-12-10T21:13',
 'dateModified': ''}

In [54]:
html = str(driver.page_source.encode('utf-8'))
re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', html).group(1)

'2018-12-12T23:02'

In [15]:
with open('/Users/Nozomi/Desktop/a.html','w', encoding='utf-8') as f:
    f.write(str(html))